In [1]:
from os import listdir, environ
from sys import path

import pandas as pd

import openai
from openai.error import RateLimitError, InvalidRequestError
import backoff

import json
import ast

In [2]:
openai.organization = environ.get('OPEN_AI_ORG')
openai.api_key = environ.get('OPEN_AI_API_KEY')

In [3]:
@backoff.on_exception(backoff.expo, RateLimitError)
def extract_paragraph_info_json_three(paragraph):
    messages = []
    
    messages.append({"role": "system", "content":'''# who you are: you are helpful assistant, expert in chemistry.
    # your task: extract synthesis parameters from scientific text.
    # you will take input in format: #####<paragraph>#####
    # you will respond in json format: 
    {"synthesis_approach": [<value/values (separated by comma)> or '' (empty string)], "optimization_method": [<value/values> or ''], "formulations": [<value/values> or ''], "precursor": [<value/values> or ''], "concentration_&_volume_of_precursor_solutions": [<value/values> or ''], "reduction_agents": [<value/values> or ''], "stabilizing_agent": [<value/values> or ''], "stabilizing_agent_type": [<value/values> or ''], "stabilizing_agent_concentration": [<value/values> or ''], "mixing_&_reaction_conditions": [<value/values> or ''], "mass_ratio": [<value/values> or ''], "precautions": [<value/values> or ''], "storage": [<value/values> or ''], "post-preparation_analysis": [<value/values> or '']}'''})
    
    messages.append({"role": "user", "content": '#####'+paragraph+'#####'})
    
    response = openai.ChatCompletion.create(model="gpt-4-turbo-preview", 
                                            messages=messages, max_tokens=4095, temperature=1)

    # Print the response and add it to the messages list
    chat_message = response['choices'][0]['message']['content']
    # print(f"Bot: {chat_message}")
    # messages.append({"role": "assistant", "content": chat_message})
    return(chat_message)

In [4]:
df_paragraphs = pd.read_pickle('SeNp_synth_paragraphs_labeled.pkl')

In [5]:
df_paragraphs_synth = \
df_paragraphs[df_paragraphs['label_cleaned_splited'] == 'YES']

In [6]:
df_paragraphs_synth['synth_prams_json_three'] = \
df_paragraphs_synth['section_text_cleaned_splited']\
.map(lambda x: extract_paragraph_info_json_three(x))

/tmp/ipykernel_41645/2443598559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paragraphs_synth['synth_prams_json_three'] = \


In [7]:
df_paragraphs_synth['synth_prams_json_three']

0      ```json\n{\n  "synthesis_approach": ["foam rep...
17     {"synthesis_approach": ["mixing aqueous soluti...
17     ```json\n{\n  "synthesis_approach": ["Spray dr...
17     ```json\n{\n  "synthesis_approach": [""],\n  "...
25     ```json\n{\n  "synthesis_approach": ["Bio-synt...
25     ```json\n{\n  "synthesis_approach": ["Jain et ...
29     ```json\n{\n  "synthesis_approach": ["Reddish-...
40     ```json\n{\n  "synthesis_approach": ["chemical...
70     ```json\n{\n  "synthesis_approach": ["Mixing, ...
70     ```json\n{\n  "synthesis_approach": ["reaction...
71     ```json\n{\n  "synthesis_approach": ["radiatio...
78     ```json\n{\n  "synthesis_approach": ["Wet chem...
78     ```json\n{\n    "synthesis_approach": ["dissol...
86     {"synthesis_approach": ["function method"], "o...
92     ```json\n{\n  "synthesis_approach": ["green sy...
99     ```json\n{\n  "synthesis_approach": ["reductio...
108    ```json\n{\n  "synthesis_approach": ["reductio...
108    ```json\n{\n    "synthes

In [8]:
pd.DataFrame(df_paragraphs_synth['synth_prams_json_three']\
.map(lambda x: json.loads(x.replace('json', '').replace('```', ''))).tolist())

,synthesis_approach,optimization_method,formulations,precursor,concentration_&_volume_of_precursor_solutions,reduction_agents,stabilizing_agent,stabilizing_agent_type,stabilizing_agent_concentration,mixing_&_reaction_conditions,mass_ratio,precautions,storage,post-preparation_analysis
0,"[foam replica method, physicochemical solvent/...",[],"[45S5Bioglass® powder (45% SiO2, 24.5% CaO, 24...","[ascorbic acid, sodium selenite, bovine serum ...",[],[ascorbic acid],[bovine serum albumin (BSA)],[],[],"[dissolving ascorbic acid in distilled water, ...",[],[],[],[]
1,[mixing aqueous solutions],[],"[HAuCl4 (3 mM), NaSeSO4 (2 mM)]","[HAuCl4, NaSeSO4]","[HAuCl4 (3 mM), NaSeSO4 (2 mM); volume ratio o...",[],[],[],[],"[stirred at room temperature, monitored every ...",[],"[precipitated via centrifugation, washed two t...",[],[UV/Vis spectrophotometry]
2,[Spray drying],,[BSA nanospheres with Au2Se/Au nanoparticles],"[BSA powder, Au2Se/Au nanoparticles]",[0.25 g of BSA in 25 mL of distilled water],,[BSA],[Protein],,"[Heating to 150 °C, Maintained at 150 °C for 3 h]","[BSA to Au2Se/Au: 50:1, BSA to Au2Se/Au: 50:3,...",,,
3,[],[],[(Au2Se/Au and ZnPc)-loaded BSA nanospheres (2...,"[HNO3, HClO4]","[HNO3 (5 mL), HClO4 (1 mL)]",[],[BSA],[],[],"[HNO3 treated at 100 °C for 2 h, HClO4 treated...",[],[],[],[inductively coupled plasma–atomic emission sp...
4,[Bio-synthesis],,[Bio-synthesized elemental selenium nanopartic...,[Sodium selenite],"[10 mM, 100 mL Luria-Bertani (LB) broth]",[Comamonas testosteroni S44],,,,"[Incubation at 28 °C, shaking at 150 rpm for 3...",,"[Centrifugation for 10 min at 8,000 rpm, Sonic...",[Temporarily preserving at -20 °C],[]
5,[Jain et al. protocol with minor modifications],,"[sodium selenite, L-reduced glutathione (GSH)]",[sodium selenite],[200 µL 1 M sodium selenite],[L-reduced glutathione (GSH)],,,,"[room temperature, French pressure for lysis]",[0.25 g GSH],[Cells collected by centrifugation and washed ...,,
6,[Reddish-orange homogeneous colloid],[],[Cts-Se colloid],"[low molecular weight Cts, ascorbic acid, sodi...","[0.15 g in 50 ml double distilled water, 25 ml...",[ascorbic acid],"[low molecular weight Cts, acetic acid]","[polymer, acid]",[some drops of 1.0% acetic acid solution],[Stirring at room temperature],[],[],[],"[UV-vis spectrophotometry, Particle size analy..."
7,[chemical reduction],[DOE methodology],"[SeNPs-BSA, SeNPs-Chit, SeNPs-Gluc]",[Sodium selenite],[0.02 M (12.5 mL) for SeNPs-BSA and SeNPs-Chit...,"[Ascorbic acid for SeNPs-BSA and SeNPs-Chit, G...","[BSA, Chitosan]","[Protein for BSA, Polysaccharide for Chitosan]",[0.87% w/w (5 mL)],[Ascorbic acid + stabilizer solutions mix then...,[1:1 between sodium selenite and chitosan or BSA],[Reaction vessels covered with aluminum foil t...,[4–8◦C],"[FTIR, XRD]"
8,"[Mixing, Refluxing]",[],"[Sodium selenosulphate solution, PVA stock sol...","[Selenium powder, Na2SO3]",[Selenium powder (2 g) and Na2SO3 (20 g) in 10...,[],[PVA],[Polymer],[1%],"[Refluxed at 70 °C for 7 h, Stirring at 80 °C ...","[Selenium powder:Na2SO3 = 2g:20g, PVA:Water = ...",[Solution kept in dark to prevent photo-oxidat...,[],[]
9,[reaction of sodium selenosulphate with differ...,"[studying the reaction kinetics, spectrophotom...",[PVA-stabilized Se nanoparticles],[sodium selenosulphate],[5x10^-4 to 1.5x10^-3 mol dm^-3],[],[PVA],[polymer],[0.05–0.15%],"[aqueous medium, different time periods depend...",[],[separation of selenium nanoparticles using hi...,[],"[spectrophotometric method, acid test for conf..."


In [9]:
df_paragraphs_synth['synth_prams_json_three_from_raw'] = \
df_paragraphs_synth['sections']\
.map(lambda x: extract_paragraph_info_json_three(x.get('text')))

/tmp/ipykernel_41645/3461017776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paragraphs_synth['synth_prams_json_three_from_raw'] = \


In [12]:
df_jsons = pd.DataFrame(df_paragraphs_synth['synth_prams_json_three']\
.map(lambda x: json.loads(x.replace('json', '').replace('```', ''))).tolist())

In [14]:
df_jsons.to_csv('extracted_synth_parameters.csv', index=False)

In [23]:
df_jsons_protein = \
pd.concat([df_paragraphs_synth[['title']].reset_index(drop=True),
df_jsons[['synthesis_approach', 'precursor',	
'reduction_agents',	'stabilizing_agent', 'stabilizing_agent_type']].reset_index(drop=True)], axis=1)

In [47]:
df_jsons_protein[\
(df_jsons_protein['stabilizing_agent'].map(lambda x: 'BSA' in ','.join(x))) | \
(df_jsons_protein['stabilizing_agent'].map(lambda x: 'PVA' in ','.join(x))) | \
(df_jsons_protein['stabilizing_agent'].map(lambda x: 'protein' in ','.join(x)))]

,title,synthesis_approach,precursor,reduction_agents,stabilizing_agent,stabilizing_agent_type
0,45S5Bioglass®-based scaffolds coated with sele...,"[foam replica method, physicochemical solvent/...","[ascorbic acid, sodium selenite, bovine serum ...",[ascorbic acid],[bovine serum albumin (BSA)],[]
2,Bovine Serum Albumin Nanospheres Synchronously...,[Spray drying],"[BSA powder, Au2Se/Au nanoparticles]",,[BSA],[Protein]
3,Bovine Serum Albumin Nanospheres Synchronously...,[],"[HNO3, HClO4]",[],[BSA],[]
7,Comparative Study of the Antimicrobial Activit...,[chemical reduction],[Sodium selenite],"[Ascorbic acid for SeNPs-BSA and SeNPs-Chit, G...","[BSA, Chitosan]","[Protein for BSA, Polysaccharide for Chitosan]"
8,Vinyl monomers-induced synthesis of polyvinyl ...,"[Mixing, Refluxing]","[Selenium powder, Na2SO3]",[],[PVA],[Polymer]
9,Vinyl monomers-induced synthesis of polyvinyl ...,[reaction of sodium selenosulphate with differ...,[sodium selenosulphate],[],[PVA],[polymer]
10,Vinyl monomers-induced synthesis of polyvinyl ...,[radiation-induced method],[sodium selenosulphate],[],[PVA],[polymer]
14,Mediated by tea polypeptides: A green synthesi...,[green synthesis],[tea protein],[tea protein],[tea protein],[plant-derived polypeptides]
16,The Influence of Synthesis Conditions on the A...,[reduction],[Na2SeO3],[ascorbic acid (AA)],[polyvinyl alcohol (PVA)],[polymer]
17,The Influence of Synthesis Conditions on the A...,"[Cys as reductant, PVA as stabilizing agent]",[Na2SeO3],[Cys],[PVA],


In [48]:
df_jsons_protein = \
pd.concat([df_paragraphs_synth[['title']].reset_index(drop=True),
df_jsons[['synthesis_approach', 'stabilizing_agent']].reset_index(drop=True)], axis=1)

In [49]:
df_jsons_protein[\
(df_jsons_protein['stabilizing_agent'].map(lambda x: 'BSA' in ','.join(x))) | \
(df_jsons_protein['stabilizing_agent'].map(lambda x: 'PVA' in ','.join(x))) | \
(df_jsons_protein['stabilizing_agent'].map(lambda x: 'protein' in ','.join(x)))]

,title,synthesis_approach,stabilizing_agent
0,45S5Bioglass®-based scaffolds coated with sele...,"[foam replica method, physicochemical solvent/...",[bovine serum albumin (BSA)]
2,Bovine Serum Albumin Nanospheres Synchronously...,[Spray drying],[BSA]
3,Bovine Serum Albumin Nanospheres Synchronously...,[],[BSA]
7,Comparative Study of the Antimicrobial Activit...,[chemical reduction],"[BSA, Chitosan]"
8,Vinyl monomers-induced synthesis of polyvinyl ...,"[Mixing, Refluxing]",[PVA]
9,Vinyl monomers-induced synthesis of polyvinyl ...,[reaction of sodium selenosulphate with differ...,[PVA]
10,Vinyl monomers-induced synthesis of polyvinyl ...,[radiation-induced method],[PVA]
14,Mediated by tea polypeptides: A green synthesi...,[green synthesis],[tea protein]
16,The Influence of Synthesis Conditions on the A...,[reduction],[polyvinyl alcohol (PVA)]
17,The Influence of Synthesis Conditions on the A...,"[Cys as reductant, PVA as stabilizing agent]",[PVA]
